In [11]:
#Data set class

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return "NEGATIVE"
        elif self.score == 3:
            return "NEUTRAL"
        else: #score of 4 or 5
            return "POSITIVE"
        
        

In [14]:
import json

file_name = './data/sentiment/Books_small.json'

#take of the differece(reviews,review)
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall'])) #reviewText(text) and overall(score)
        
reviews[5].text

'Love the book, great story line, keeps you entertained.for a first novel from this author she did a great job,  Would definitely recommend!'